In [9]:
# imports
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import itertools

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Network Architecture
num_inputs = 5
num_hidden = 3
mid_size = 2
num_outputs = 4

# Temporal Dynamics
num_steps = 10
beta = 0.95

# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, mid_size)
        self.lif2 = snn.Leaky(beta=beta)
        self.fc3 = nn.Linear(mid_size, num_outputs)
        self.lif3 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        
        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        for step in range(num_steps):
            cur1 = self.fc1(x)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)
        
# Load the network onto CUDA if available
net = Net().to(device)
        
    # print(layer)

# # Print all modules, including nested ones (shows every submodule)
# for module in net.modules():
#     print(module)

from dev_get_torchnn_info import group_torch_layers, build_fugu_network
layer_dicts = group_torch_layers(net)
print(len(layer_dicts))
scaffold = build_fugu_network(layer_dicts, n_steps=num_steps)

print('Network edges (source → target):')
for u, v, d in scaffold.graph.edges(data=True):
    print(f'{u} → {v} | weight={d.get("weight")} | delay={d.get("delay")}')


3
Network edges (source → target):
Input-22:begin → Input-22:complete | weight=1.0 | delay=9
Input-22:begin → Layer0_begin | weight=0.0 | delay=1
Input-22:complete → Layer0_complete | weight=0.0 | delay=1
Input-22:(0,) → Layer0d0 | weight=-0.019373105838894844 | delay=1
Input-22:(0,) → Layer0d1 | weight=-0.16408365964889526 | delay=1
Input-22:(0,) → Layer0d2 | weight=-0.07293377071619034 | delay=1
Input-22:(1,) → Layer0d0 | weight=0.09173393249511719 | delay=1
Input-22:(1,) → Layer0d1 | weight=0.12741155922412872 | delay=1
Input-22:(1,) → Layer0d2 | weight=-0.08976095914840698 | delay=1
Input-22:(2,) → Layer0d0 | weight=0.11953038722276688 | delay=1
Input-22:(2,) → Layer0d1 | weight=0.29423001408576965 | delay=1
Input-22:(2,) → Layer0d2 | weight=-0.2128874510526657 | delay=1
Input-22:(3,) → Layer0d0 | weight=0.33817917108535767 | delay=1
Input-22:(3,) → Layer0d1 | weight=-0.21992549300193787 | delay=1
Input-22:(3,) → Layer0d2 | weight=0.03668986260890961 | delay=1
Input-22:(4,) → Layer